In [3]:
import sys 
sys.path.insert(1, "../")
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import ast

import dask.dataframe as dd
from dask.dataframe import from_pandas

from workloads.util import use_results, use_dataset, assigned_query_timestamps, join_queries_features

%load_ext autoreload
%autoreload 2


# TODO
# 1. write timestamps out at ingest time 
# pretend queries run at that ingest time 

In [4]:
experiment = "stl-yahoo-A4-keys-100-interval-10000-events-200000-queries-200000"

results_dir = use_results(experiment)
dataset_dir = use_dataset(experiment, redownload=True)
print(results_dir)
print(dataset_dir)

/data/wooders/ralf-vldb//datasets/stl-yahoo-A4-keys-100-interval-10000-events-200000-queries-200000
/data/wooders/ralf-vldb//results/stl-yahoo-A4-keys-100-interval-10000-events-200000-queries-200000
/data/wooders/ralf-vldb//datasets/stl-yahoo-A4-keys-100-interval-10000-events-200000-queries-200000


In [5]:
queries_df = pd.read_csv(f"{dataset_dir}/queries.csv")
oracle_df = pd.read_csv(f"{dataset_dir}/oracle_features_672.csv").dropna()
oracle_df["seasonality"] = oracle_df['seasonality'].dropna().apply(ast.literal_eval)

In [6]:
result_file = "results_workers_8_fifo_window_672_slide_672.csv"
timestamp_file = "results_workers_8_fifo_window_672_slide_672_timestamps.csv"
result_df = pd.read_csv(f"{results_dir}/{result_file}")
timestamp_df = pd.read_csv(f"{results_dir}/{timestamp_file}")

In [7]:
timestamp_df["timestamp_rel"] = timestamp_df["timestamp"] -  timestamp_df["timestamp"].min()

In [71]:
timestamp_df 

,timestamp_ms,timestamp,timestamp_rel
0,0,1.645404e+09,0.000000
1,1,1.645404e+09,0.047005
2,2,1.645404e+09,0.049488
3,3,1.645404e+09,0.051473
4,4,1.645404e+09,0.053289
...,...,...,...
9725,9725,1.645404e+09,102.897656
9726,9726,1.645404e+09,102.937619
9727,9727,1.645404e+09,102.939794
9728,9728,1.645404e+09,102.941704


In [87]:
queries_df

,Unnamed: 0,key_id,value,timestamp_ms,query_id
0,0,1,4.946131,0,0
1,54000,28,-5.811925,0,54000
2,56000,29,-14.364553,0,56000
3,58000,30,1.951412,0,58000
4,60000,31,-4.561680,0,60000
...,...,...,...,...,...
199995,139999,70,3214.956142,9995,139999
199996,141999,71,851.205020,9995,141999
199997,143999,72,3297.483037,9995,143999
199998,147999,74,-1021.602950,9995,147999


In [8]:
queries_df = queries_df.merge(timestamp_df, on="timestamp_ms")

In [9]:
queries_df["timestamp_rel"] = queries_df["timestamp"] -  timestamp_df["timestamp"].min()
queries_df

,Unnamed: 0,key_id,value,timestamp_ms,query_id,timestamp,timestamp_rel
0,0,1,4.946131,0,0,1.645944e+09,0.000000
1,54000,28,-5.811925,0,54000,1.645944e+09,0.000000
2,56000,29,-14.364553,0,56000,1.645944e+09,0.000000
3,58000,30,1.951412,0,58000,1.645944e+09,0.000000
4,60000,31,-4.561680,0,60000,1.645944e+09,0.000000
...,...,...,...,...,...,...,...
194495,177944,89,1963.828935,9720,177944,1.645944e+09,25.139877
194496,115944,58,1047.246245,9720,115944,1.645944e+09,25.139877
194497,165944,83,-3257.870528,9720,165944,1.645944e+09,25.139877
194498,35944,18,4704.447669,9720,35944,1.645944e+09,25.139877


In [10]:
oracle_df

,key_id,timestamp_ms,Unnamed: 0,trend,noise,outlier,seasonality,value,timestamp_ms.1,key_id.1
671,1,3355,671,2016.469410,3.244026,0.0,"[-144.98071706444813, -8.069063105050581, 138....",1867.598891,3355,1
672,1,3360,672,2019.651941,2.723627,0.0,"[-7.0755059309315165, 138.3620714201056, 257.4...",2021.723627,3360,1
673,1,3365,673,2022.792336,0.920390,0.0,"[151.21166402922807, 257.4088253121081, 326.23...",2174.565525,3365,1
674,1,3370,674,2025.982974,-0.449394,0.0,"[268.0948497450916, 326.28930830970796, 336.81...",2295.670898,3370,1
675,1,3375,675,2029.129638,8.419333,0.0,"[339.0043113748398, 336.76588909530074, 261.67...",2370.769283,3375,1
...,...,...,...,...,...,...,...,...,...,...
199995,100,9975,1995,3042.264443,-29.601895,-144.0,"[40.26943347580342, -89.4103524274973, -142.62...",3029.208512,9975,100
199996,100,9980,1996,3045.475374,-36.791996,-146.0,"[38.83311947332407, -144.03280623371666, -207....",3019.727482,9980,100
199997,100,9985,1997,3051.419224,66.016470,-157.0,"[60.128920888434536, -209.55885963897197, 105....",3087.048742,9985,100
199998,100,9990,1998,3058.540377,-15.802843,151.0,"[148.32137067378477, 104.54253114279629, 57.75...",3273.209377,9990,100


In [11]:
oracle_df = oracle_df.merge(timestamp_df, on="timestamp_ms")

In [12]:
oracle_df

,key_id,timestamp_ms,Unnamed: 0,trend,noise,outlier,seasonality,value,timestamp_ms.1,key_id.1,timestamp,timestamp_rel
0,1,3355,671,2016.469410,3.244026,0.0,"[-144.98071706444813, -8.069063105050581, 138....",1867.598891,3355,1,1.645944e+09,8.700255
1,2,3355,671,-812.454989,0.806107,0.0,"[-150.14269497711325, 38.790663608293436, 121....",-927.585500,3355,2,1.645944e+09,8.700255
2,3,3355,671,1348.087792,-30.543072,0.0,"[-380.36793334635405, -60.91572026707621, 454....",949.291162,3355,3,1.645944e+09,8.700255
3,4,3355,671,-599.868897,-2.087670,0.0,"[-260.92362549760617, 6.3124958680008, 224.007...",-801.060896,3355,4,1.645944e+09,8.700255
4,5,3355,671,-1.499277,114.364495,0.0,"[-298.1762183228843, 53.862303751243935, 255.0...",-226.565215,3355,5,1.645944e+09,8.700255
...,...,...,...,...,...,...,...,...,...,...,...,...
127395,96,9720,1944,-2607.915120,-2.094158,127.0,"[107.36803031756433, 10.12792921015005, 456.83...",-2439.870960,9720,96,1.645944e+09,25.139877
127396,97,9720,1944,5367.537112,7.551763,-209.0,"[-221.5331032715197, 237.97681987822062, 438.5...",5128.547736,9720,97,1.645944e+09,25.139877
127397,98,9720,1944,653.773963,-84.627416,-225.0,"[-260.6655037646143, 291.20482332271837, 342.7...",176.626739,9720,98,1.645944e+09,25.139877
127398,99,9720,1944,-1379.679353,-8.293898,-29.0,"[-81.65959344242073, -0.221054718061918, 170.8...",-1450.541743,9720,99,1.645944e+09,25.139877


In [13]:
queries_df.merge(oracle_df, on=["timestamp_ms", "key_id", "value"]).sort_values("timestamp") #, lsuffix='_query', rsuffix='_oracle')

KeyError: 'timestamp'

In [ ]:
df = join_queries_features(queries_df, result_df)

In [ ]:
final_df = queries_df.merge(df, on=["key_id", "timestamp_ms"], how="left")

In [ ]:
final_df[final_df["key_id"] == 3].dropna()

In [103]:
oracle_df[oracle_df["key_id"] == 3]

,key_id,timestamp_ms,Unnamed: 0,trend,noise,outlier,seasonality,value,timestamp_ms.1,key_id.1,timestamp,timestamp_rel
2,3,3355,671,1348.087792,-30.543072,0.0,"[-380.36793334635405, -60.91572026707621, 454....",949.291162,3355,3,1.645404e+09,34.400306
102,3,3360,672,1351.020220,-54.285209,0.0,"[-51.25232386902484, 455.4206331033498, 686.52...",1291.714791,3360,3,1.645404e+09,34.458172
202,3,3365,673,1352.334681,19.756825,0.0,"[387.1258021089658, 687.052438096461, 809.3801...",1731.922591,3365,3,1.645404e+09,34.512284
302,3,3370,674,1353.708153,13.208731,0.0,"[672.982109756391, 809.487265454404, 903.91396...",2019.759892,3370,3,1.645404e+09,34.566360
402,3,3375,675,1355.553144,-9.533808,0.0,"[798.7279753712465, 903.726999481986, 736.3793...",2165.573101,3375,3,1.645404e+09,34.629125
...,...,...,...,...,...,...,...,...,...,...,...,...
127002,3,9705,1941,3891.471485,33.235114,178.0,"[-753.7482239793538, -947.8178504734859, -298....",3186.229910,9705,3,1.645404e+09,102.691215
127102,3,9710,1942,3896.935698,88.877187,174.0,"[-575.7658630460937, -299.8488540013164, -340....",3378.644245,9710,3,1.645404e+09,102.753230
127202,3,9715,1943,3897.021561,66.589137,151.0,"[-375.3252746894086, -338.5300603657268, -8.07...",3600.114302,9715,3,1.645404e+09,102.802596
127302,3,9720,1944,3899.734320,-13.741041,-142.0,"[-254.38787786094204, -8.173708907529388, 607....",3565.517438,9720,3,1.645404e+09,102.850083


In [44]:
result_df["timestamp"] = result_df["processing_time"] - result_df["processing_time"].min()

In [45]:
result_df[result_df["key_id"] == 23].sort_values("processing_time")

,key_id,trend,seasonality,timestamp_ms,processing_time,runtime,ingest_time,timestamp
10,23,"[-196.22011254707834, -170.65317703914403, -14...","[-249.5235835438441, 785.8515120244592, 661.66...",6345,1.645403e+09,0.105917,1.645403e+09,1.650257
179,23,"[-224.83878762855878, -250.04811806717444, -27...","[-71.44447419937666, 5.682705043955988, 161.34...",6870,1.645403e+09,0.111515,1.645403e+09,7.353309
208,23,"[990.9024138306361, 910.7789179825783, 831.533...","[-38.46451366120079, -116.71428955747679, -390...",7010,1.645403e+09,0.099837,1.645403e+09,8.132111
387,23,"[-426.29817600447745, -400.45221027263466, -37...","[-164.66266995119216, -186.15890160616743, 306...",7410,1.645403e+09,0.105905,1.645403e+09,13.147302
406,23,"[-1101.03673121283, -1047.1710919952973, -994....","[-13.937811796747413, 261.1407150249671, 371.0...",7440,1.645403e+09,0.103427,1.645403e+09,13.612861
575,23,"[-236.9026690642196, -281.45503812938824, -325...","[-113.82555799907632, -321.24304815604967, -32...",7885,1.645403e+09,0.106703,1.645403e+09,18.304016
578,23,"[-641.4514062744807, -685.0824487759766, -727....","[76.34803387161256, -32.31876305094539, -248.5...",7960,1.645403e+09,0.099920,1.645403e+09,18.379168
692,23,"[-1337.5579621003512, -1333.4687396677587, -13...","[92.53999187026572, -346.10360516082346, -337....",8345,1.645403e+09,0.101212,1.645403e+09,21.496841
780,23,"[-1289.678855355192, -1281.2438663793748, -127...","[421.58391203036126, 104.39041626381835, -76.3...",8415,1.645403e+09,0.104852,1.645403e+09,23.815234
940,23,"[-984.9311525662611, -1013.0754933665883, -104...","[422.3763943378006, -506.81648213725697, -1077...",8970,1.645403e+09,0.110468,1.645403e+09,28.100254


In [39]:
joined_df = pd.read_csv(f"{results_dir}/query_{result_file}")

In [40]:
joined_df

,query_id,key_id,trend,seasonality,timestamp_ms,processing_time,runtime,ingest_time,timestamp,query_key_id,query_timestamp_ms,query_timestamp
0,1990,1,"[1903.4719981702765, 1897.8804668770274, 1892....","[158.6950451335515, 201.78987671788843, 90.087...",9755,1.645403e+09,0.103572,1.645403e+09,1.645403e+09,1.0,9950.0,1.645413e+09
1,1991,1,"[1903.4719981702765, 1897.8804668770274, 1892....","[158.6950451335515, 201.78987671788843, 90.087...",9755,1.645403e+09,0.103572,1.645403e+09,1.645403e+09,1.0,9955.0,1.645413e+09
2,1992,1,"[1903.4719981702765, 1897.8804668770274, 1892....","[158.6950451335515, 201.78987671788843, 90.087...",9755,1.645403e+09,0.103572,1.645403e+09,1.645403e+09,1.0,9960.0,1.645413e+09
3,1993,1,"[1903.4719981702765, 1897.8804668770274, 1892....","[158.6950451335515, 201.78987671788843, 90.087...",9755,1.645403e+09,0.103572,1.645403e+09,1.645403e+09,1.0,9965.0,1.645413e+09
4,1994,1,"[1903.4719981702765, 1897.8804668770274, 1892....","[158.6950451335515, 201.78987671788843, 90.087...",9755,1.645403e+09,0.103572,1.645403e+09,1.645403e+09,1.0,9970.0,1.645413e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
995,199995,100,"[1651.3229779534179, 1658.4516032422998, 1665....","[-57.779720064371965, -26.703197551159626, 29....",9370,1.645403e+09,0.100735,1.645403e+09,1.645403e+09,100.0,9975.0,1.645413e+09
996,199996,100,"[1651.3229779534179, 1658.4516032422998, 1665....","[-57.779720064371965, -26.703197551159626, 29....",9370,1.645403e+09,0.100735,1.645403e+09,1.645403e+09,100.0,9980.0,1.645413e+09
997,199997,100,"[1651.3229779534179, 1658.4516032422998, 1665....","[-57.779720064371965, -26.703197551159626, 29....",9370,1.645403e+09,0.100735,1.645403e+09,1.645403e+09,100.0,9985.0,1.645413e+09
998,199998,100,"[1651.3229779534179, 1658.4516032422998, 1665....","[-57.779720064371965, -26.703197551159626, 29....",9370,1.645403e+09,0.100735,1.645403e+09,1.645403e+09,100.0,9990.0,1.645413e+09


In [41]:
joined_df[joined_df["key_id"] == 23].sort_values(by="query_id")

,query_id,key_id,trend,seasonality,timestamp_ms,processing_time,runtime,ingest_time,timestamp,query_key_id,query_timestamp_ms,query_timestamp
220,45990,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9950.0,1.645413e+09
221,45991,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9955.0,1.645413e+09
222,45992,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9960.0,1.645413e+09
223,45993,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9965.0,1.645413e+09
224,45994,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9970.0,1.645413e+09
225,45995,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9975.0,1.645413e+09
226,45996,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9980.0,1.645413e+09
227,45997,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9985.0,1.645413e+09
228,45998,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9990.0,1.645413e+09
229,45999,23,"[-942.1633228790918, -936.0040274948295, -930....","[-241.97788464687108, 1047.9597221068775, 456....",9695,1.645403e+09,0.103063,1.645403e+09,1.645403e+09,23.0,9995.0,1.645413e+09


In [105]:
start_time = result_df["processing_time"].min()
start_time

1645404341.7159903

In [107]:
result_df["timestamp_rel"] = result_df["timestamp"] - timestamp_df["timestamp"].min()
result_df

,key_id,trend,seasonality,timestamp_ms,processing_time,runtime,ingest_time,timestamp,timestamp_rel
0,26,"[824.2751275093282, 740.453435808707, 657.1962...","[46.552118721690945, 45.669387765395506, 53.15...",6240,1.645404e+09,0.301992,1.645404e+09,1.645404e+09,66.795285
1,50,"[991.6476235482272, 907.5561527813163, 824.591...","[-40.74052961443177, 195.64677336846756, 321.1...",6380,1.645404e+09,0.109238,1.645404e+09,1.645404e+09,68.066183
2,45,"[650.431854034271, 607.5653533402942, 566.0588...","[-9.884058456236927, 209.0377694011591, -480.6...",6405,1.645404e+09,0.110232,1.645404e+09,1.645404e+09,68.355348
3,68,"[829.4474187149448, 789.5238682499357, 749.895...","[-524.2692506623875, -215.88990120660196, -646...",6410,1.645404e+09,0.307485,1.645404e+09,1.645404e+09,68.600431
4,8,"[226.00856375396697, 215.85967199330523, 206.1...","[752.5128217799784, 181.04366695737366, 274.25...",6425,1.645404e+09,0.100551,1.645404e+09,1.645404e+09,68.705076
...,...,...,...,...,...,...,...,...,...
1345,27,"[-363.7938449487854, -307.13372948576875, -252...","[-196.330974101205, -780.9134160242694, -1709....",9880,1.645404e+09,0.099257,1.645404e+09,1.645404e+09,106.912487
1346,9,"[-1109.042555869865, -1096.8228535222536, -108...","[281.81128527797784, -240.5450933448294, -346....",9885,1.645404e+09,0.099009,1.645404e+09,1.645404e+09,106.971308
1347,88,"[-478.29569500409315, -555.8158248965194, -630...","[594.0898068505018, -795.440518964438, -93.497...",9865,1.645404e+09,0.100059,1.645404e+09,1.645404e+09,106.976220
1348,38,"[777.6404318910774, 802.6610993394191, 827.699...","[-67.70481802614242, 579.5639753633312, 381.36...",9960,1.645404e+09,0.099348,1.645404e+09,1.645404e+09,106.985643


In [54]:
q_df = assigned_query_timestamps(queries_df, start_time, interval_time=0.001)

0            0
1            0
2            0
3            0
4            0
          ... 
199995    9995
199996    9995
199997    9995
199998    9995
199999    9995
Name: timestamp_ms, Length: 200000, dtype: int64
timestamp 0         1.645403e+09
1         1.645403e+09
2         1.645403e+09
3         1.645403e+09
4         1.645403e+09
              ...     
199995    1.645403e+09
199996    1.645403e+09
199997    1.645403e+09
199998    1.645403e+09
199999    1.645403e+09
Name: timestamp, Length: 200000, dtype: float64


In [55]:
q_df["timtestamp_rel"] = q_df["timestamp"] - result_df["processing_time"].min()
q_df[q_df["key_id"] == 23]

,Unnamed: 0,key_id,value,timestamp_ms,query_id,timestamp,timtestamp_rel
46,44000,23,113.662456,0,44000,1.645403e+09,0.000
106,44001,23,404.928971,5,44001,1.645403e+09,0.005
211,44002,23,873.986911,10,44002,1.645403e+09,0.010
362,44003,23,1020.966895,15,44003,1.645403e+09,0.015
435,44004,23,1174.382461,20,44004,1.645403e+09,0.020
...,...,...,...,...,...,...,...
199505,45995,23,-357.701023,9975,45995,1.645403e+09,9.975
199667,45996,23,-194.787311,9980,45996,1.645403e+09,9.980
199725,45997,23,-608.738903,9985,45997,1.645403e+09,9.985
199832,45998,23,-1399.918479,9990,45998,1.645403e+09,9.990


In [16]:
queries_df[queries_df["key_id"] == 23]

,Unnamed: 0,key_id,value,timestamp_ms,query_id
46,44000,23,113.662456,0,44000
106,44001,23,404.928971,5,44001
211,44002,23,873.986911,10,44002
362,44003,23,1020.966895,15,44003
435,44004,23,1174.382461,20,44004
...,...,...,...,...,...
199505,45995,23,-357.701023,9975,45995
199667,45996,23,-194.787311,9980,45996
199725,45997,23,-608.738903,9985,45997
199832,45998,23,-1399.918479,9990,45998


In [9]:
joined_df.timestamp.max()

1645172511.9025602

In [12]:
queries_df.timestamp_ms.value_counts()

0       100
6230    100
2005    100
5975    100
3540    100
       ... 
2475    100
4520    100
170     100
6185    100
8060    100
Name: timestamp_ms, Length: 2000, dtype: int64

In [20]:
df = queries_df.sort_values(["key_id", "timestamp_ms"])

In [21]:
df[df["key_id"] == 23]

,Unnamed: 0,key_id,value,timestamp_ms,query_id
46,44000,23,113.662456,0,44000
106,44001,23,404.928971,5,44001
211,44002,23,873.986911,10,44002
362,44003,23,1020.966895,15,44003
435,44004,23,1174.382461,20,44004
...,...,...,...,...,...
199505,45995,23,-357.701023,9975,45995
199667,45996,23,-194.787311,9980,45996
199725,45997,23,-608.738903,9985,45997
199832,45998,23,-1399.918479,9990,45998
